In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop, SGD
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy import misc, ndimage
import glob
import matplotlib.image as mpimg
import cv2

#from transform_image_data import centre_crop

%matplotlib inline

In [2]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
print(tf.__version__)

Default GPU Device:/device:GPU:0
2.1.0


In [3]:
#import Data

train_folder = "C:/Users/Hameez/.conda/envs/gemstone_dataset/Data/Cropped/Train/"
test_folder = "C:/Users/Hameez/.conda/envs/gemstone_dataset/Data/Test/"


# img = image.load_img(train_folder+"/Alexandrite/alexandrite_2.jpg")
# # plt.imshow(img)
# # plt.show()
# print(cv2.imread("Data/Train/Alexandrite/alexandrite_24.jpg").shape)

#------------Image augmentation for train images----------------------------------#
train = ImageDataGenerator(rescale=1/255, shear_range=0.5,
                                  rotation_range=90, channel_shift_range=2.5)
#---------------------------------------------------------------------------#

test = ImageDataGenerator(rescale=1/255)

im_height = im_width = 224

train_data = train.flow_from_directory(train_folder,
                                       target_size=(im_height,im_width),
                                       class_mode='categorical')
test_data = test.flow_from_directory(test_folder,
                                       target_size=(im_height,im_width),
                                       class_mode='categorical')





Found 2857 images belonging to 87 classes.
Found 363 images belonging to 87 classes.


In [4]:
# images = []
# stone = 'Ametrine'
# for img_path in glob.glob(train_folder+stone+'/*.jpg'):
#     images.append(mpimg.imread(img_path))

# plt.figure(figsize=(25,25))
# columns = 4
# for i, image in enumerate(images):
#     if i > 7:
#         break
#     else:
#         plt.subplot(len(images) / columns + 1, columns, i + 1)
#         plt.title(stone)
#         plt.imshow(image)
        

In [5]:
# test_augment = ImageDataGenerator(rescale=1/255, zoom_range=2.0, shear_range=0.5,
#                                   rotation_range=60, channel_shift_range=2.5)#, ,)
# img = image.load_img(train_folder+"/Chalcedony/chalcedony_5.jpg")
# plt.imshow(img)



In [6]:

# x = img_to_array(x)

# x = x.reshape((1,) + x.shape)
# # x.shape

# i = 0
# for batch in test_augment.flow(x, batch_size=1,
#                               save_to_dir='Augmented', save_prefix='alexandrite',
#                                save_format='jpeg',):
#     i += 1
#     if i > 20:
#         break

### CNN Model 

In [7]:

cnn_model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(im_height,im_width,3)),
                                        tf.keras.layers.BatchNormalization(),
                                        tf.keras.layers.MaxPool2D(2,2),
                                        #
                                        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                        tf.keras.layers.BatchNormalization(),
                                        tf.keras.layers.MaxPool2D(2,2),
                                        
                                        tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
                                        tf.keras.layers.BatchNormalization(),
                                        tf.keras.layers.MaxPool2D(2,2),
                                        
                                        tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
                                        tf.keras.layers.BatchNormalization(),
                                        tf.keras.layers.MaxPool2D(2,2),
                                        #
#                                         tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
#                                         tf.keras.layers.MaxPool2D(2,2),
                                        #
                                        tf.keras.layers.Flatten(),
                                        ##
                                        tf.keras.layers.Dense(512, activation='relu'),
                                        tf.keras.layers.Dropout(0.1),
                                        #
                                        tf.keras.layers.Dense(512, activation='relu'),
                                        tf.keras.layers.Dropout(0.1),
                                        #
                                        tf.keras.layers.Dense(512, activation='relu'),
                                        tf.keras.layers.Dropout(0.1),
                                        #
                                        tf.keras.layers.Dense(87, activation='softmax')
                                        ])
cnn_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 109, 109, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       7

In [ ]:
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
cnn_model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

history = cnn_model.fit_generator(train_data, epochs=50, validation_data=test_data, verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 12 steps
Epoch 1/50
90/90 [==============================] - 78s 871ms/step - loss: 3.6270 - accuracy: 0.1481 - val_loss: 4.6383 - val_accuracy: 0.0138
Epoch 2/50
90/90 [==============================] - 72s 801ms/step - loss: 2.5934 - accuracy: 0.2930 - val_loss: 5.1393 - val_accuracy: 0.0220
Epoch 3/50
90/90 [==============================] - 167s 2s/step - loss: 2.2237 - accuracy: 0.3626 - val_loss: 5.3063 - val_accuracy: 0.0275
Epoch 4/50
90/90 [==============================] - 100s 1s/step - loss: 2.0158 - accuracy: 0.4106 - val_loss: 4.2428 - val_accuracy: 0.0579
Epoch 5/50
90/90 [==============================] - 140s 2s/step - loss: 1.8229 - accuracy: 0.4491 - val_loss: 3.3015 - val_accuracy: 0.1570
Epoch 6/50
90/90 [==============================] - 124s 1s/step - loss: 1.6960 - accuracy: 0.4844 - val_loss: 3.4187 - val

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-89c75f9f1d70>", line 6, in <module>
    history = cnn_model.fit_generator(train_data, epochs=50, validation_data=test_data, verbose=1)
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 1306, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 342, in

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-377a4e210667>", line 2, in <module>
    plt.plot(history.history['accuracy'])
NameError: name 'history' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Use

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ERROR! Session/line number was not unique in database. History logging moved to new session 1023


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-d737d78baf69>", line 2, in <module>
    plt.plot(history.history['loss'])
NameError: name 'history' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\H

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-d737d78baf69>", line 2, in <module>
    plt.plot(history.history['loss'])
NameError: name 'history' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Hameez\.conda\envs\gpu_tf\lib\site-packages\IPython\core\interactiveshell.py", line 3338, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\Users\Hameez\.co

In [ ]:

model_path ='Saved Models/gs_mcl_v3.SGD_10ep.h5'
cnn_model.save(model_path)

model = tf.keras.models.load_model(model_path)

In [ ]:
shape = (im_height,im_width)
unsean_stone_path = "C:/Users/Hameez/.conda/envs/gemstone_dataset/Data/Unseen/rutilated-quartz-zoom.jpg"
    
# #img = cv2.imread(os.path.join(unsean_stones_folder)
img = image.load_img(unsean_stone_path, target_size=shape)
#new_img = centre_crop(img, 25)
img_arr = image.img_to_array(img)
print(os.path.basename(unsean_stone_path))
img

In [ ]:
img_arr = np.expand_dims(img_arr, axis=0)
prediction = model.predict_classes(img_arr)
classess = train_data.class_indices
for name in classess:
    if classess[name] == int(prediction):
        print("Predicts that its a: "+name)
    else:
        "Couldn't find stone name!"
